# Downloading and building cells from Allen Cell Types Database

- [Allen Cell Types Database](http://celltypes.brain-map.org/data)
- [Downloading biophysical models](https://allensdk.readthedocs.io/en/latest/biophysical_models.html)
- [Example on building cells](https://alleninstitute.github.io/AllenSDK/_static/examples/nb/pulse_stimulus.html)

In [1]:
!pip install --quiet allensdk neuron

In [1]:
from allensdk.api.queries.biophysical_api import BiophysicalApi
from allensdk.model.biophys_sim.config import Config
from allensdk.model.biophysical.utils import Utils

## Download the cell

Use `specimen_id` to view models available for that specimen.

In [ ]:
bp = BiophysicalApi()

# Examples
# --------
#
# Mouse L2/3 SST (Som) cell
# http://celltypes.brain-map.org/mouse/experiment/electrophysiology/476686112
#
# Mouse L5 cell
# https://celltypes.brain-map.org/experiment/electrophysiology/480351780

bp.get_neuronal_models(480351780)

[{'id': 482529696,
  'name': 'Biophysical - perisomatic_Rorb-IRES2-Cre-D;Ai14-197851.04.01.01',
  'neuron_reconstruction_id': 491770985,
  'neuronal_model_template_id': 329230710,
  'specimen_id': 480351780},
 {'id': 497232429,
  'name': 'Biophysical - all active_Rorb-IRES2-Cre-D;Ai14-197851.04.01.01',
  'neuron_reconstruction_id': 491770985,
  'neuronal_model_template_id': 491455321,
  'specimen_id': 480351780}]

Use cell id (`id`) to download the cell data to `working_directory`.

In [ ]:
# Download the cell
bp.cache_stimulus = False # False = do not download the large stimulus NWB file
bp.cache_data(497232429, working_directory = 'L5') # The cell will be downloaded to `working_directory`

2025-07-04 17:14:26,944 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/491770987
2025-07-04 17:14:27,425 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/496605092
2025-07-04 17:14:27,654 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/395337019
2025-07-04 17:14:27,862 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/395337225
2025-07-04 17:14:28,049 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/395337054
2025-07-04 17:14:28,251 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/395337070
2025-07-04 17:14:28,431 allensdk.api.api.retrieve_fi

## Build the cell

In [ ]:
# Go to `working_directory`
%cd /content/L5

# Compile the modfiles
!nrnivmodl modfiles

# Create the h object
description = Config().load('manifest.json')
utils = Utils(description)
h = utils.h

/content/SOM
/usr/local/bin/nrnivmodl:10: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import working_set
/content/SOM
Mod files: "modfiles/modfiles/CaDynamics.mod" "modfiles/modfiles/Ca_HVA.mod" "modfiles/modfiles/Ca_LVA.mod" "modfiles/modfiles/Ih.mod" "modfiles/modfiles/Im.mod" "modfiles/modfiles/Im_v2.mod" "modfiles/modfiles/Kd.mod" "modfiles/modfiles/K_P.mod" "modfiles/modfiles/K_T.mod" "modfiles/modfiles/Kv2like.mod" "modfiles/modfiles/Kv3_1.mod" "modfiles/modfiles/Nap.mod" "modfiles/modfiles/NaTa.mod" "modfiles/modfiles/NaTs.mod" "modfiles/modfiles/NaV.mod" "modfiles/modfiles/SK.mod"

Creating 'x86_64' directory for .o files.

 -> Compiling mod_func.cpp
 -> NMODL ../modfiles/Ca_HVA.mod
 -> NMODL ../modfiles/CaDynamics.mod
 -> NMODL ../modfiles/Ca_LVA.mod
Translating Ca_HVA.mod into /content/SOM/x86_64/Ca_HVA.c
Thread Safe
 -> NMODL ../modfiles/Ih.mod
Translating Ca_LVA.mod into /content

In [6]:
# Configure morphology
manifest = description.manifest
morphology_path = description.manifest.get_path('MORPHOLOGY')
utils.generate_morphology(morphology_path.encode('ascii', 'ignore'))
utils.load_cell_parameters()

warning ./Sst-IRES-Cre_Ai14-188740.03.02.01_657210399_m.swc: more than one tree:
  root at line 4
  root at line 14504
  root at line 14725
  root at line 14798
  root at line 15141
  root at line 15144
  root at line 15156
  root at line 15211
  root at line 15318
  root at line 15496
  root at line 15811
  root at line 15857
  root at line 15883
  root at line 16161
  root at line 16200
  root at line 16210
  root at line 16253
  root at line 16286
  root at line 16301
  root at line 16326
  root at line 16574
  root at line 16613
  root at line 16618
  root at line 16645
  root at line 16695
  root at line 16751
  root at line 16765
  root at line 16768
